# Business Question: 
An entrepreneur decided to use his wealth to create a kind of NGO to help the local population become more aware of how to behave at sea, in order to avoid shark attacks (Putting up signs, creating a free app with information, giving local lectures, etc.)

The problem is, this person does not know where to start and only has enough capital to serve one region/state.

So the question is, where should he/she start?

# Analysis:

First step was to import all the libs that I thought about using and the csv itself, also setting the number of columns and
rows I wanted to show.

In [1]:
import pandas as pd
import numpy as np
import regex as re
pd.set_option('display.max_columns', 24)
pd.set_option('display.max_rows', 20)
df_attacks = pd.read_csv('attacks.csv')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 698: invalid start byte

Simply importing the csv resulted in a "UnicodeDecodeError", so I used the function "open" to see the csv's encoding and add that parameter to the read_csv function

In [2]:
open('attacks.csv')

<_io.TextIOWrapper name='attacks.csv' mode='r' encoding='cp1252'>

In [3]:
df_attacks = pd.read_csv('attacks.csv', encoding='cp1252')

df_attacks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25723 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             8702 non-null   object 
 1   Date                    6302 non-null   object 
 2   Year                    6300 non-null   float64
 3   Type                    6298 non-null   object 
 4   Country                 6252 non-null   object 
 5   Area                    5847 non-null   object 
 6   Location                5762 non-null   object 
 7   Activity                5758 non-null   object 
 8   Name                    6092 non-null   object 
 9   Sex                     5737 non-null   object 
 10  Age                     3471 non-null   object 
 11  Injury                  6274 non-null   object 
 12  Fatal (Y/N)             5763 non-null   object 
 13  Time                    2948 non-null   object 
 14  Species                 3464 non-null 

To make things easier to work with, I started by changing the name of the columns.

In [8]:
#changed the columns names to lowercase and underline instead of spaces and the column "Fatal (Y/N)" was changed to "fatal"
new_columns = [column_name.strip().lower() for column_name in df_attacks.columns]
df_attacks.columns = new_columns
df_attacks = df_attacks.rename(columns = {'fatal (y/n)': 'fatal'})
df_attacks.columns = [re.sub(' ', '_', column).lower() for column in df_attacks.columns]
df_attacks.columns

Index(['case_number', 'date', 'year', 'type', 'country', 'area', 'location',
       'activity', 'name', 'sex', 'age', 'injury', 'fatal', 'time', 'species',
       'investigator_or_source', 'pdf', 'href_formula', 'href',
       'case_number.1', 'case_number.2', 'original_order', 'unnamed:_22',
       'unnamed:_23'],
      dtype='object')

The next steps were to drop columns and rows that had too many null values, also filtering the data to the rows that mattered for my analysis, ending up in a new dataframe named "df_clean" after the drops.

In [9]:
#first I removed the duplicates, which cleared most of the columns with too many null values
df_droplines = df_attacks.drop_duplicates()
df_droplines.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6312 entries, 0 to 25722
Data columns (total 24 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   case_number             6310 non-null   object 
 1   date                    6302 non-null   object 
 2   year                    6300 non-null   float64
 3   type                    6298 non-null   object 
 4   country                 6252 non-null   object 
 5   area                    5847 non-null   object 
 6   location                5762 non-null   object 
 7   activity                5758 non-null   object 
 8   name                    6092 non-null   object 
 9   sex                     5737 non-null   object 
 10  age                     3471 non-null   object 
 11  injury                  6274 non-null   object 
 12  fatal                   5763 non-null   object 
 13  time                    2948 non-null   object 
 14  species                 3464 non-null  

In [11]:
#here I droped all columns that had more than 3000 null values
null_columns = df_droplines.isnull().sum()
dropcolumns = list(null_columns[null_columns > 3000].index)
df_dropcol = df_droplines.drop(dropcolumns, axis=1)
df_dropcol

,case_number,date,year,type,country,area,location,activity,name,sex,age,injury,fatal,species,investigator_or_source,pdf,href_formula,href,case_number.1,case_number.2,original_order
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6307,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6309.0
6308,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6310.0
6309,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
#now I filtered the dataframe to show only the columns that mattered for the analysis
df_ctm = df_dropcol[['type', 'country', 'year', 'area', 'age', 'location', 'activity', 'fatal']]
df_ctm

,type,country,year,area,age,location,activity,fatal
0,Boating,USA,2018.0,California,57,"Oceanside, San Diego County",Paddling,N
1,Unprovoked,USA,2018.0,Georgia,11,"St. Simon Island, Glynn County",Standing,N
2,Invalid,USA,2018.0,Hawaii,48,"Habush, Oahu",Surfing,N
3,Unprovoked,AUSTRALIA,2018.0,New South Wales,NaN,Arrawarra Headland,Surfing,N
4,Provoked,MEXICO,2018.0,Colima,NaN,La Ticla,Free diving,N
...,...,...,...,...,...,...,...,...
6307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6308,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6309,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#checking number of nulls
null_columns2 = df_ctm.isnull().sum()
null_columns2

type          14
country       60
year          12
area         465
age         2841
location     550
activity     554
fatal        549
dtype: int64

In [14]:
#eliminate rows with null values in country/area/location/activity simultaneously
df_clean = df_ctm[df_ctm.country.notnull() & df_ctm.area.notnull() & df_ctm.location.notnull() & df_ctm.activity.notnull()]
df_clean

,type,country,year,area,age,location,activity,fatal
0,Boating,USA,2018.0,California,57,"Oceanside, San Diego County",Paddling,N
1,Unprovoked,USA,2018.0,Georgia,11,"St. Simon Island, Glynn County",Standing,N
2,Invalid,USA,2018.0,Hawaii,48,"Habush, Oahu",Surfing,N
3,Unprovoked,AUSTRALIA,2018.0,New South Wales,NaN,Arrawarra Headland,Surfing,N
4,Provoked,MEXICO,2018.0,Colima,NaN,La Ticla,Free diving,N
...,...,...,...,...,...,...,...,...
6290,Unprovoked,SOUTH AFRICA,0.0,KwaZulu-Natal,NaN,Durban,Wading,Y
6296,Unprovoked,AUSTRALIA,0.0,New South Wales,NaN,,Swimming,Y
6297,Unprovoked,AUSTRALIA,0.0,Western Australia,NaN,Roebuck Bay,Diving,Y
6299,Unprovoked,USA,0.0,North Carolina,NaN,Ocracoke Inlet,Swimming,Y


I noticed that the data before 1900 were only about 12% of the data and judged it was too old and probably not precise enough, so I filtered my data to only show events after 1900.

In [15]:
df_clean = df_clean[df_clean.year >= 1900]
df_clean

,type,country,year,area,age,location,activity,fatal
0,Boating,USA,2018.0,California,57,"Oceanside, San Diego County",Paddling,N
1,Unprovoked,USA,2018.0,Georgia,11,"St. Simon Island, Glynn County",Standing,N
2,Invalid,USA,2018.0,Hawaii,48,"Habush, Oahu",Surfing,N
3,Unprovoked,AUSTRALIA,2018.0,New South Wales,NaN,Arrawarra Headland,Surfing,N
4,Provoked,MEXICO,2018.0,Colima,NaN,La Ticla,Free diving,N
...,...,...,...,...,...,...,...,...
5558,Unprovoked,CROATIA,1900.0,Primorje-Gorski Kotar County,NaN,"Volosko, Opatija",Swimming,Y
5559,Invalid,USA,1900.0,Hawaii,NaN,"Makapu'u Point, O'ahu",Hunting seashells,NaN
5560,Provoked,USA,1900.0,Connecticut,NaN,"Bridgeport, Fairfield County",,N
5561,Unprovoked,AUSTRALIA,1900.0,New South Wales,NaN,"Lane Cove River, Sydney Harbor (Estuary)","Standing, gathering oysters",N


On the next steps I checked which countries had the most attacks and if, in those countries, there was an expecific area with more attacks or if the attacks were spread throughout the country.
That way I could decide the "where" (I would start the company efforts).

In [16]:
print(len(df_clean)) #total incidents 
print(len(df_clean['country'].value_counts())) #number of countries
df_clean['country'].value_counts().head(20) #incidents/country

4653
132


USA                 1932
AUSTRALIA           1057
SOUTH AFRICA         503
BRAZIL                96
PAPUA NEW GUINEA      93
NEW ZEALAND           90
BAHAMAS               63
MEXICO                63
ITALY                 50
REUNION               38
MOZAMBIQUE            37
PHILIPPINES           35
NEW CALEDONIA         33
FIJI                  32
SPAIN                 32
CUBA                  24
EGYPT                 23
FRENCH POLYNESIA      21
CROATIA               20
PANAMA                17
Name: country, dtype: int64

In [17]:
df_usa = df_clean[df_clean.country == 'USA'] #incidents only in the USA
print(len(df_usa)) #number os incidents in the USA
df_usa['area'].value_counts().head(20) #incidents/area(state)

1932


Florida              937
California           273
Hawaii               261
South Carolina       120
North Carolina        86
Texas                 63
New Jersey            38
Oregon                28
Virginia              16
New York              15
Georgia               12
Massachusetts         11
Alabama               11
Puerto Rico           10
Louisiana             10
Delaware               7
Maryland               5
US Virgin Islands      5
Mississippi            4
Connecticut            4
Name: area, dtype: int64

With the filters I used above, I could tell 2 things:
1) There are 132 countries registered in this data.

2) Out of those 132 countries, 80% of the incidents happened in only ~4% of the countries.

3) The obvious choice was Florida, since, out of the 10 countries in which 80% of the incidents occured, 25% of those incidents happened in Florida (almost as much incidents that happened in all Australia).

Now that the "where" is decided, we can work with the "what" and maybe the "who" (what are the most vulnerable activities and if theres a specific age group that does that activity and is attacked)

In [19]:
#this step was to filter the incidents in Florida and treating the activities names to make them easier to work with
for columns in df_usa.columns:
    df_usa['area'] = df_usa['area'].str.strip()
    df_usa['activity'] = df_usa['activity'].str.lower() 
    
df_florida = df_usa[df_usa.area == 'Florida']
df_florida

C:\Users\thoma\AppData\Local\Temp\ipykernel_23268\1155439524.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_usa['area'] = df_usa['area'].str.strip()
C:\Users\thoma\AppData\Local\Temp\ipykernel_23268\1155439524.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_usa['activity'] = df_usa['activity'].str.lower()


,type,country,year,area,age,location,activity,fatal
7,Unprovoked,USA,2018.0,Florida,52,"Lighhouse Point Park, Ponce Inlet, Volusia County",fishing,N
8,Unprovoked,USA,2018.0,Florida,15,"Cocoa Beach, Brevard County",walking,N
9,Unprovoked,USA,2018.0,Florida,12,"Daytona Beach, Volusia County",standing,N
61,Unprovoked,USA,2017.0,Florida,14,"Floridana Beach, Brevard County",surfing,N
69,Unprovoked,USA,2017.0,Florida,17,"Juno Beach, Palm Beach County",surfing,N
...,...,...,...,...,...,...,...,...
5417,Unprovoked,USA,1909.0,Florida,16,"Panama City, Bay County",swimming,N
5457,Unprovoked,USA,1907.0,Florida,NaN,"Garden Key, Charlotte County","fishing, tarpon being chased by shark leapt ...",Y
5488,Provoked,USA,1905.0,Florida,NaN,"Fort Pierce, St Lucie County",fishing,N
5496,Unprovoked,USA,1905.0,Florida,NaN,"Pablo Beach, Jacksonville, Duval County",swimming,N


In [21]:
#list of activities in a descending order of occurrences
acts = df_florida.groupby('activity').count()[['area']]
acts.sort_values('area', ascending=False).head(20)

,area
activity,
surfing,364
swimming,131
wading,67
standing,32
fishing,31
spearfishing,24
boogie boarding,17
walking,10
body boarding,10


Since a lot of activities are similar, I decided to put them together by similarity, i.e. 'windsurfing' was considered the same as 'surfing'.

In [22]:
def activ(x):
    if "fish" in x:
        return "fishing"
    elif "surf" in x or "board" in x:
        return "surfing"
    elif "swim" in x or "bath" in x:
        return "swimming"
    elif "wading" in x or "wade" in x:
        return "wading"
    elif "diving" in x or "dive" in x:
        return "diving"
    elif "stand" in x:
        return "standing"
    elif "snorkel" in x:
        return "snorkeling"
    else:
        return "other"

In [23]:
df_florida["activity"] = df_florida["activity"].map(activ)

df_florida

C:\Users\thoma\AppData\Local\Temp\ipykernel_23268\849924034.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_florida["activity"] = df_florida["activity"].map(activ)


,type,country,year,area,age,location,activity,fatal
7,Unprovoked,USA,2018.0,Florida,52,"Lighhouse Point Park, Ponce Inlet, Volusia County",fishing,N
8,Unprovoked,USA,2018.0,Florida,15,"Cocoa Beach, Brevard County",other,N
9,Unprovoked,USA,2018.0,Florida,12,"Daytona Beach, Volusia County",standing,N
61,Unprovoked,USA,2017.0,Florida,14,"Floridana Beach, Brevard County",surfing,N
69,Unprovoked,USA,2017.0,Florida,17,"Juno Beach, Palm Beach County",surfing,N
...,...,...,...,...,...,...,...,...
5417,Unprovoked,USA,1909.0,Florida,16,"Panama City, Bay County",swimming,N
5457,Unprovoked,USA,1907.0,Florida,NaN,"Garden Key, Charlotte County",fishing,Y
5488,Provoked,USA,1905.0,Florida,NaN,"Fort Pierce, St Lucie County",fishing,N
5496,Unprovoked,USA,1905.0,Florida,NaN,"Pablo Beach, Jacksonville, Duval County",swimming,N


In [24]:
acts2 = df_florida.groupby('activity').count()[['area']]
acts2.sort_values('area', ascending=False).head(20)

,area
activity,
surfing,456
swimming,150
fishing,97
other,78
wading,71
diving,43
standing,34
snorkeling,9


With the last analysis we can see that out of the 938 incidents, and a total of "8" activities, 3 of them are more vulnerable: surfing, swimming and fishing, that will be the company focus (the "what").

That leaves us to finding out the "who".

In [27]:
#filtering the Florida incidents in which the activity was either surfing, swimming or fishing
df_age = df_florida[df_florida['activity'].str.contains("surfing" or "swimming" or "fishing")]
df_age

,type,country,year,area,age,location,activity,fatal
61,Unprovoked,USA,2017.0,Florida,14,"Floridana Beach, Brevard County",surfing,N
69,Unprovoked,USA,2017.0,Florida,17,"Juno Beach, Palm Beach County",surfing,N
77,Unprovoked,USA,2017.0,Florida,NaN,"Hanna Park, Jacksonville, Duval County",surfing,N
80,Unprovoked,USA,2017.0,Florida,NaN,Fort Worth,surfing,N
81,Unprovoked,USA,2017.0,Florida,NaN,Palm Beach County,surfing,N
...,...,...,...,...,...,...,...,...
3632,Unprovoked,USA,1965.0,Florida,17,"Juno Beach, Palm Beach County",surfing,N
3763,Invalid,USA,1963.0,Florida,NaN,Off Key West,surfing,NaN
3947,Invalid,USA,1960.0,Florida,51,"Hutchinson Island Beach, Martin County",surfing,NaN
5116,Invalid,USA,1928.0,Florida,NaN,Off Florida coast,surfing,NaN


The following steps removed lines where age had a null value or something inaccurate. This removed a significant portion of data (~12%), so that needs to be taken into account, but it was necessary for the analysis, since I was grouping the incidents by the age of the victims.

In [29]:
df_age = df_age[df_age.age.notnull()]
df_age['age'].info()

<class 'pandas.core.series.Series'>
Int64Index: 408 entries, 61 to 3947
Series name: age
Non-Null Count  Dtype 
--------------  ----- 
408 non-null    object
dtypes: object(1)
memory usage: 6.4+ KB


In [30]:
df_age_drop = df_age[pd.to_numeric(df_age.age,errors='coerce').notnull()]
df_age_drop['age'].info()

<class 'pandas.core.series.Series'>
Int64Index: 400 entries, 61 to 3947
Series name: age
Non-Null Count  Dtype 
--------------  ----- 
400 non-null    object
dtypes: object(1)
memory usage: 6.2+ KB


In [31]:
df_age_drop['age'] = df_age_drop['age'].astype('float64')

C:\Users\thoma\AppData\Local\Temp\ipykernel_23268\2242444142.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_age_drop['age'] = df_age_drop['age'].astype('float64')


In [32]:
df_age_drop['age'].info()

<class 'pandas.core.series.Series'>
Int64Index: 400 entries, 61 to 3947
Series name: age
Non-Null Count  Dtype  
--------------  -----  
400 non-null    float64
dtypes: float64(1)
memory usage: 6.2 KB


After cleaning the dataframe again, I separated the age column into bins, to create "age groups".

In [34]:
Bin = pd.cut(df_age_drop.age, 5)
df_age_drop['age_bin'] = Bin
df_age_drop

C:\Users\thoma\AppData\Local\Temp\ipykernel_23268\1737492850.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_age_drop['age_bin'] = Bin


,type,country,year,area,age,location,activity,fatal,age_bin
61,Unprovoked,USA,2017.0,Florida,14.0,"Floridana Beach, Brevard County",surfing,N,"(5.92, 22.0]"
69,Unprovoked,USA,2017.0,Florida,17.0,"Juno Beach, Palm Beach County",surfing,N,"(5.92, 22.0]"
83,Unprovoked,USA,2017.0,Florida,28.0,"Ponce Inlet, Volusia County",surfing,N,"(22.0, 38.0]"
87,Unprovoked,USA,2017.0,Florida,18.0,"Ponce Inlet, Volusia County",surfing,N,"(5.92, 22.0]"
95,Unprovoked,USA,2017.0,Florida,17.0,"New Smyrna Beach, Volusia County",surfing,N,"(5.92, 22.0]"
...,...,...,...,...,...,...,...,...,...
3424,Unprovoked,USA,1969.0,Florida,13.0,"St. Petersburg, Pinnellas County",surfing,N,"(5.92, 22.0]"
3480,Unprovoked,USA,1968.0,Florida,20.0,"Jensen Beach, Martin County",surfing,N,"(5.92, 22.0]"
3482,Unprovoked,USA,1968.0,Florida,17.0,Palm Beach County,surfing,N,"(5.92, 22.0]"
3632,Unprovoked,USA,1965.0,Florida,17.0,"Juno Beach, Palm Beach County",surfing,N,"(5.92, 22.0]"


In [35]:
df_age_drop.age_bin.value_counts().sort_index()

(5.92, 22.0]    246
(22.0, 38.0]    105
(38.0, 54.0]     40
(54.0, 70.0]      8
(70.0, 86.0]      1
Name: age_bin, dtype: int64

Analysing the age bins we can see that approximately 88% of the victims that were either surfing, swimming or fishing (and were attacked), were under 38 years old, and 62% were under 22 years old.

# Conclusion:

After the analysis I concluded that the company efforts should be focused on:
1) Where: Florida, where the majorite of the attacks happen.
2) What: Surfing, swimming and fishing are the most affected activities.
3) Who: 0-38 (or 0-22, depending on the focus range) are the most vulnarable age groups.*

*Note: The age group could be narrowed down to a smaller group if we had the data that showed a relation between activity and practitioners' age, to focus more precisely on a particular age group.